In [10]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random
import time
import math
import numpy as np
import matplotlib.pyplot as plt
import heapq
import wandb
plt.switch_backend('agg')
import matplotlib.ticker as ticker

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [11]:
SOS_token = 0
EOS_token = 1
MAX_LENGTH = 30

In [12]:
class Lang:
    def __init__(self, name):
        self.name = name
        self.char2index = {}
        self.char2count = {}
        self.index2char = {0: "SOS", 1: "EOS"}
        self.n_chars = 2  # Count SOS and EOS

    def addWord(self, word):
        for char in word:
            self.addChar(char)

    def addChar(self, char):
        if char not in self.char2index:
            self.char2index[char] = self.n_chars
            self.char2count[char] = 1
            self.index2char[self.n_chars] = char
            self.n_chars += 1
        else:
            self.char2count[char] += 1

In [13]:
def readLangs(lang, type, reverse=False):
    print("Reading lines...")

    # Read the file and split into lines
    
    lines = open('aksharantar_sampled/%s/%s_%s.csv' % (lang, lang, type), encoding='utf-8').read().strip().split('\n')
    
    # Split every line into pairs and normalize
    # pairs = [[normalizeString(s) for s in l.split(',')] for l in lines]
    pairs = [[s for s in l.split(',')] for l in lines]

    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang)
        output_lang = Lang('eng')
    else:
        input_lang = Lang('eng')
        output_lang = Lang(lang)

    return input_lang, output_lang, pairs

In [14]:
def prepareData(lang, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang, reverse)
    print("Read %s word pairs" % len(pairs))
    print("Trimmed to %s word pairs" % len(pairs))
    print("Counting chars...")
    for pair in pairs:
        input_lang.addWord(pair[0])
        output_lang.addWord(pair[1])
    print("Counted chars:")
    print(input_lang.name, input_lang.n_chars)
    print(output_lang.name, output_lang.n_chars)
    return input_lang, output_lang, pairs


input_lang, output_lang, pairs = prepareData('mni', 'train')
print(random.choice(pairs))

Reading lines...
Read 10060 word pairs
Trimmed to 10060 word pairs
Counting chars...
Counted chars:
eng 28
mni 46
['ahumlak', 'ꯑꯍꯨꯝꯂꯛ']


In [15]:
cells = {'rnn': nn.RNN, 'gru': nn.GRU, 'lstm': nn.LSTM}

In [16]:
class EncoderRNN(nn.Module):
    """
    Class for the encoder RNN.
    """
    def __init__(self, input_size, embedding_size, hidden_size, num_hidden_layers, dropout=0.2, cell_type='gru'):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_hidden_layers = num_hidden_layers
        self.embedding = nn.Embedding(input_size, embedding_size)
        self.cell_type = cell_type
        self.cell = cells[cell_type](embedding_size, hidden_size, num_hidden_layers, dropout=dropout)

    def forward(self, input, hidden, cell=None):
        embedded = self.embedding(input).view(1, 1, -1)
        if self.cell_type == 'rnn' or self.cell_type == 'gru':
            output, hidden = self.cell(embedded, hidden)
            return output, hidden
        else:
            output, (hidden, cell) = self.cell(embedded, (hidden, cell))    
            return output, hidden, cell

    def initHidden(self):
        return torch.zeros(self.num_hidden_layers, 1, self.hidden_size, device=device)

In [17]:
class BeamSearchNode:
    def __init__(self, decoder_output, hidden, prev_node, char_idx, log_prob, length):
        self.decoder_output = decoder_output
        self.hidden = hidden
        self.prev_node = prev_node
        self.char_idx = char_idx
        self.log_prob = log_prob
        self.length = length

    # def __lt__(self, other):
    #     return self.log_prob > other.log_prob

    def eval(self):
        return self.log_prob / self.length

In [18]:
class DecoderRNN(nn.Module):
    """
    Class for the vanilla decoder RNN.
    """
    def __init__(self, hidden_size, embedding_size, output_size, num_hidden_layers, dropout=0, cell_type='gru'):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_hidden_layers = num_hidden_layers
        self.embedding = nn.Embedding(output_size, embedding_size)
        self.cell_type = cell_type
        self.cell = cells[cell_type](embedding_size, hidden_size, num_hidden_layers, dropout=dropout)
        self.out = nn.Linear(hidden_size, output_size)
        self.output_size = output_size
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden, cell=None):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        if self.cell_type == 'rnn' or self.cell_type == 'gru':
            output, hidden = self.cell(output, hidden)
        else:
            output, (hidden, cell) = self.cell(output, (hidden, cell))
            
        output = self.softmax(self.out(output[0]))

        if self.cell_type == 'rnn' or self.cell_type == 'gru':
            return output, hidden
        else:
            return output, hidden, cell
    
    def initHidden(self):
        return torch.zeros(self.num_hidden_layers, 1, self.hidden_size, device=device)

    def beam_search(self, encoder_outputs, decoder_hidden, beam_width, max_length, start_token, end_token):
        # Initialize the beam search
        beam_nodes = [BeamSearchNode(None, decoder_hidden, None, start_token, 0, 1)]
        done_nodes = []

        # Keep expanding the beam until we reach the maximum length or all candidates are done
        for _ in range(max_length):
            candidates = []
            for node in beam_nodes:
                if node.char_idx == end_token:
                    done_nodes.append(node)
                    continue

                # Feed the previous char and hidden state into the decoder
                output, hidden = self.forward(torch.tensor([node.char_idx]), node.hidden)
                # Single depth or double depth for tensor?

                # Generate new candidate nodes and add them to the heap
                for char_idx in range(self.output_size):
                    length = node.length + 1
                    log_prob = node.log_prob + output[0][char_idx].item()
                    new_node = BeamSearchNode(output, hidden, node, char_idx, log_prob, length)
                    heapq.heappush(candidates, (-new_node.eval(), id(new_node), new_node))

            # Select the top k candidates to continue expanding the beam
            beam_nodes = []
            for _ in range(beam_width):
                if not candidates:
                    break
                _, _, node = heapq.heappop(candidates)
                beam_nodes.append(node)

            if not beam_nodes:
                break

        # Return the best candidate as the predicted sequence
        done_nodes.extend(beam_nodes)
        best_node = max(done_nodes, key=lambda node: node.eval())
        # predicted_seq = []
        # while best_node.prev_node is not None:
        #     predicted_seq.append(best_node.char_idx)
        #     best_node = best_node.prev_node
        # return predicted_seq[::-1]
        return best_node

In [19]:
def indexesFromWord(lang, word):
    return [lang.char2index[char] for char in word]


def tensorFromWord(lang, word):
    indexes = indexesFromWord(lang, word)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromWord(input_lang, pair[0])
    target_tensor = tensorFromWord(output_lang, pair[1])
    return (input_tensor, target_tensor)

In [20]:
teacher_forcing_ratio = 0.5


def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, beam_width, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    if encoder.cell_type == 'lstm':
        encoder_cell = encoder.initHidden()

    loss = 0

    for ei in range(input_length):
        # print(cell_type)
        if encoder.cell_type == 'lstm':
            encoder_output, encoder_hidden, encoder_cell = encoder(input_tensor[ei], encoder_hidden, encoder_cell)
        else:
            encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_hidden = encoder_hidden

    if beam_width == 1:
        decoder_input = torch.tensor([[SOS_token]], device=device)

        if decoder.cell_type == 'lstm':
            decoder_cell = encoder_cell

        use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

        if use_teacher_forcing:
            # Teacher forcing: Feed the target as the next input
            for di in range(target_length):
                # decoder_output, decoder_hidden, decoder_attention = decoder(decoder_input, decoder_hidden, encoder_outputs)
                if decoder.cell_type == 'lstm':
                    decoder_output, decoder_hidden, decoder_cell = decoder(decoder_input, decoder_hidden, decoder_cell)
                else:
                    decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
                # print("Criterion:", criterion(decoder_output, target_tensor[di])) 
                loss += criterion(decoder_output, target_tensor[di])
                # print(loss)
                decoder_input = target_tensor[di]  # Teacher forcing

        else:
            # Without teacher forcing: use its own predictions as the next input
            for di in range(target_length):
                # decoder_output, decoder_hidden, decoder_attention = decoder(decoder_input, decoder_hidden, encoder_outputs)
                if decoder.cell_type == 'lstm':
                    decoder_output, decoder_hidden, decoder_cell = decoder(decoder_input, decoder_hidden, decoder_cell)
                else:
                    decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
                topv, topi = decoder_output.topk(1)
                decoder_input = topi.squeeze().detach()  # detach from history as input

                # print("Criterion:", criterion(decoder_output, target_tensor[di])) 
                loss += criterion(decoder_output, target_tensor[di])
                # print(loss)
                if decoder_input.item() == EOS_token:
                    break

        loss.backward()

        encoder_optimizer.step()
        decoder_optimizer.step()

        return loss.item() / target_length
    
    else:
        best_node = decoder.beam_search(encoder_outputs, decoder_hidden, beam_width, target_length, SOS_token, EOS_token)

        decoder_outputs = []
        curr_node = best_node

        while curr_node.decoder_output is not None:
            decoder_outputs.append(curr_node.decoder_output)
            curr_node = curr_node.prev_node

        decoder_outputs = decoder_outputs[::-1]

        # Loss is the sum of the criterion of each decoder output with respect to the target
        loss = sum([criterion(decoder_output, target_tensor[i]) for i, decoder_output in enumerate(decoder_outputs)])
        # print(loss)
        loss.backward()

        encoder_optimizer.step()
        decoder_optimizer.step()

        return loss.item() / target_length


In [21]:
def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [22]:
def trainIters(encoder, decoder, n_iters, optimizer=optim.SGD, cell_type='gru', beam_width=1, print_every=1000, plot_every=100, learning_rate=5e-3):
    start = time.time()
    print_loss_total = 0  # Reset every print_every

    encoder_optimizer = optimizer(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optimizer(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(pairs)) for _ in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder, decoder, cell_type, encoder_optimizer, decoder_optimizer, criterion, beam_width)
        print_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters), iter, iter / n_iters * 100, print_loss_avg))

In [23]:
def evaluate(encoder, decoder, word, beam_width=1, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromWord(input_lang, word)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        if encoder.cell_type == 'lstm':
            encoder_cell = encoder.initHidden()

        for ei in range(input_length):
            if encoder.cell_type == 'lstm':
                encoder_output, encoder_hidden, encoder_cell = encoder(input_tensor[ei], encoder_hidden, encoder_cell)
            else:
                encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)
                
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_hidden = encoder_hidden

        if beam_width == 1:
            decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

            if encoder.cell_type == 'lstm':
                decoder_cell = encoder_cell

            decoded_chars = ""
            # decoder_attentions = torch.zeros(max_length, max_length)

            for di in range(max_length):
                # decoder_output, decoder_hidden, decoder_attention = decoder(decoder_input, decoder_hidden, encoder_outputs)
                if decoder.cell_type == 'lstm':
                    decoder_output, decoder_hidden, decoder_cell = decoder(decoder_input, decoder_hidden, decoder_cell)
                else:
                    decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
                    
                # decoder_attentions[di] = decoder_attention.data
                topv, topi = decoder_output.data.topk(1)
                if topi.item() == EOS_token:
                    decoded_chars += '<EOS>'
                    break
                else:
                    decoded_chars += output_lang.index2char[topi.item()]

                decoder_input = topi.squeeze().detach()

            # return decoded_chars, decoder_attentions[:di + 1]
            return decoded_chars
        
        else:
            predicted_seq = decoder.beam_search(encoder_outputs, decoder_hidden, beam_width, SOS_token, EOS_token)
            return ''.join([output_lang.index2char[char_idx] for char_idx in predicted_seq])

In [24]:
input_valid_lang, output_valid_lang, valid_pairs = prepareData('mni', 'valid')
input_test_lang, output_test_lang, test_pairs = prepareData('mni', 'test')

def evalAccuracy(encoder, decoder, pairs):
    correct = 0
    for pair in pairs:
        output_word = evaluate(encoder, decoder, pair[0])
        output_word = output_word[:-5]
        if output_word == pair[1]:
            correct += 1
    return correct / len(pairs)

Reading lines...
Read 3260 word pairs
Trimmed to 3260 word pairs
Counting chars...
Counted chars:
eng 28
mni 46
Reading lines...
Read 4096 word pairs
Trimmed to 4096 word pairs
Counting chars...
Counted chars:
eng 28
mni 46


In [25]:
def trainBatch(encoder, decoder, epochs, optimizer=optim.SGD, beam_width=1, print_every=1000, learning_rate=5e-3, log=False):
    start = time.time()
    print_loss_total = 0  # Reset every print_every

    encoder_optimizer = optimizer(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optimizer(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(pair) for pair in pairs]
    criterion = nn.NLLLoss()

    # print("Training pairs:", len(training_pairs))

    for iter in range(1, len(training_pairs) * epochs + 1):
        training_pair = training_pairs[(iter - 1) % len(training_pairs)]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, beam_width)
        print_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / (len(training_pairs) * epochs)), iter, iter / (len(training_pairs) * epochs) * 100, print_loss_avg))

        if log:
            if iter % print_every == 0:
                wandb.log({"train_loss": loss})

            if iter % len(training_pairs) == 0:
                wandb.log({"train_accuracy": evalAccuracy(encoder, decoder, pairs), "val_accuracy": evalAccuracy(encoder, decoder, valid_pairs), "epoch": iter / len(training_pairs)})

In [26]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        # output_chars, attentions = evaluate(encoder, decoder, pair[0])
        output_word = evaluate(encoder, decoder, pair[0])
        # output_word = ''.join(output_chars)
        print('<', output_word)
        print('')

In [1253]:
hidden_size = 256
# embedding_size = input_lang.n_chars
embedding_size = 256
encoder1 = EncoderRNN(input_lang.n_chars, embedding_size, hidden_size, num_hidden_layers=1, cell_type='gru').to(device)
decoder1 = DecoderRNN(hidden_size, embedding_size, output_lang.n_chars, num_hidden_layers=1, cell_type='gru').to(device)

In [1254]:
# trainIters(encoder1, decoder1, n_iters=1, beam_width=5, print_every=100)

In [1255]:
trainBatch(encoder1, decoder1, epochs=4, beam_width=1, print_every=100)

0m 1s (- 9m 55s) (100 0%) 2.6452
0m 2s (- 9m 39s) (200 0%) 2.7250
0m 4s (- 9m 29s) (300 0%) 2.8955
0m 5s (- 9m 42s) (400 0%) 2.6631
0m 7s (- 9m 30s) (500 1%) 2.7041
0m 8s (- 9m 28s) (600 1%) 2.6729
0m 9s (- 9m 16s) (700 1%) 2.6407
0m 11s (- 9m 5s) (800 1%) 2.6048
0m 12s (- 8m 58s) (900 2%) 2.6655
0m 13s (- 8m 50s) (1000 2%) 2.6220
0m 15s (- 9m 4s) (1100 2%) 2.7234
0m 17s (- 9m 17s) (1200 2%) 2.6269
0m 18s (- 9m 26s) (1300 3%) 2.6035
0m 20s (- 9m 42s) (1400 3%) 2.6171
0m 22s (- 9m 53s) (1500 3%) 2.5768
0m 24s (- 9m 43s) (1600 3%) 2.5544
0m 25s (- 9m 32s) (1700 4%) 2.5816
0m 26s (- 9m 21s) (1800 4%) 2.6929
0m 27s (- 9m 12s) (1900 4%) 2.6096
0m 28s (- 9m 5s) (2000 4%) 2.5607
0m 29s (- 8m 59s) (2100 5%) 2.6524
0m 30s (- 8m 53s) (2200 5%) 2.5363
0m 32s (- 8m 48s) (2300 5%) 2.6139
0m 33s (- 8m 43s) (2400 5%) 2.5911
0m 34s (- 8m 38s) (2500 6%) 2.6466
0m 35s (- 8m 32s) (2600 6%) 2.6185
0m 36s (- 8m 28s) (2700 6%) 2.5745
0m 37s (- 8m 23s) (2800 6%) 2.5042
0m 38s (- 8m 18s) (2900 7%) 2.5636
0m 3

In [1256]:
encoder2 = EncoderRNN(input_lang.n_chars, embedding_size, hidden_size, num_hidden_layers=2, cell_type='gru').to(device)
decoder2 = DecoderRNN(hidden_size, embedding_size, output_lang.n_chars, num_hidden_layers=2, cell_type='gru').to(device)

In [1257]:
trainBatch(encoder2, decoder2, epochs=4, beam_width=1, print_every=100)

0m 2s (- 16m 10s) (100 0%) 2.7145
0m 5s (- 17m 32s) (200 0%) 2.7649
0m 7s (- 17m 31s) (300 0%) 2.9577
0m 10s (- 17m 32s) (400 0%) 2.8226
0m 12s (- 17m 8s) (500 1%) 2.8124
0m 15s (- 16m 36s) (600 1%) 2.7360
0m 17s (- 16m 19s) (700 1%) 2.7871
0m 19s (- 16m 20s) (800 1%) 2.7793
0m 22s (- 16m 18s) (900 2%) 2.8150
0m 25s (- 16m 22s) (1000 2%) 2.8402
0m 27s (- 16m 33s) (1100 2%) 2.7464
0m 32s (- 17m 31s) (1200 2%) 2.6716
0m 36s (- 18m 1s) (1300 3%) 2.7219
0m 40s (- 18m 53s) (1400 3%) 2.6794
0m 44s (- 19m 16s) (1500 3%) 2.5827
0m 48s (- 19m 30s) (1600 3%) 2.6698
0m 53s (- 20m 9s) (1700 4%) 2.6470
0m 57s (- 20m 31s) (1800 4%) 2.6580
1m 1s (- 20m 45s) (1900 4%) 2.6411
1m 5s (- 20m 55s) (2000 4%) 2.6749
1m 10s (- 21m 12s) (2100 5%) 2.6077
1m 14s (- 21m 30s) (2200 5%) 2.6726
1m 19s (- 21m 45s) (2300 5%) 2.5735
1m 23s (- 21m 57s) (2400 5%) 2.5459
1m 28s (- 22m 9s) (2500 6%) 2.6169
1m 30s (- 21m 55s) (2600 6%) 2.6146
1m 35s (- 22m 10s) (2700 6%) 2.6166
1m 40s (- 22m 19s) (2800 6%) 2.5944
1m 44s (- 

In [1258]:
evaluateRandomly(encoder1, decoder1)

> pei
= ꯄꯩ
< ꯄꯩ<EOS>

> yourakpada
= ꯌꯧꯔꯛꯄꯗ
< ꯌꯧꯔꯛꯄꯗ<EOS>

> khangnaribashi
= ꯈꯪꯅꯔꯤꯕꯁꯤ
< ꯈꯪꯍꯔꯥꯡꯅꯤꯁ<EOS>

> laagumna
= ꯂꯥꯒꯨꯝꯅ
< ꯂꯥꯒꯒꯝꯅ<EOS>

> sabitri
= ꯁꯥꯕꯤꯇ꯭ꯔꯤ
< ꯁꯥꯢꯕ꯭ꯔꯤ<EOS>

> kakchinglakpa
= ꯀꯛꯆꯤꯡꯂꯥꯛꯄ
< ꯀꯥꯢꯆꯪꯂꯛꯄꯄ<EOS>

> waaba
= ꯋꯥꯕ
< ꯋꯥꯕ<EOS>

> soitapa
= ꯁꯣꯢꯇꯄ
< ꯁꯤꯢꯇꯄ<EOS>

> apaamba
= ꯑꯄꯥꯝꯕ
< ꯑꯄꯥꯝꯕ<EOS>

> iriba
= ꯏꯔꯤꯕ
< ꯏꯔꯤꯕ<EOS>



In [1142]:
# pair = random.choice(pairs)

# print(pair[0])
# print(pair[1])

# with torch.no_grad():
#     input_tensor = tensorFromWord(input_lang, pair[0])
#     input_length = input_tensor.size()[0]
#     encoder1_hidden = encoder1.initHidden()

#     encoder1_outputs = torch.zeros(MAX_LENGTH, encoder1.hidden_size, device=device)

#     if encoder1.cell_type == 'lstm':
#         encoder1_cell = encoder1.initHidden()

#     for ei in range(input_length):
#         if encoder1.cell_type == 'lstm':
#             encoder1_output, encoder1_hidden, encoder1_cell = encoder1(input_tensor[ei], encoder1_hidden, encoder1_cell)
#         else:
#             encoder1_output, encoder1_hidden = encoder1(input_tensor[ei], encoder1_hidden)
            
#         encoder1_outputs[ei] += encoder1_output[0, 0]

#     decoder1_hidden = encoder1_hidden

#     if encoder1.cell_type == 'lstm':
#         decoder1_cell = encoder1_cell

#     decoded_chars = ""

#     # Predict the output using beam search
#     predicted_seq = decoder1.beam_search(encoder1_outputs, decoder1_hidden, 5, SOS_token, EOS_token)

#     # Convert the predicted sequence to a word
#     for char_idx in predicted_seq:
#         decoded_chars += output_lang.index2char[char_idx]

#     print(decoded_chars)

    



In [1266]:
evalAccuracy(encoder2, decoder2, valid_pairs)

0.55

In [18]:
# sweep_configuration = {
#     'method': 'bayes',
#     'name': 'bayes-sweep',
#     'metric': {
#         'name': 'val_accuracy',
#         'goal': 'maximize'
#     },
#     'parameters': {
#         'epochs': {
#             'values': [5, 10] 
#         },
#         'embedding_size': {
#             'values': [64, 128, 256] 
#         },
#         'num_hidden_layers': {
#             'values': [1, 2, 3]
#         },
#         'hidden_layer_size': {
#             'values': [64, 128, 256] 
#         },
#         'cell_type': {
#             'values': ['rnn', 'gru', 'lstm'] 
#         },
#         'learning_rate': {
#             'values': [1e-2, 5e-3, 1e-3, 5e-4] 
#         },
#         'optimizer': {
#             'values': ['SGD', 'Adam'] 
#         },
#         'dropout': {
#             'values': [0.0, 0.2, 0.3] 
#         },
#         'beam_width': {
#             'values': [1] #[1, 3, 5]
#         }
#     }
# }


In [18]:
sweep_configuration = {
    'method': 'bayes',
    'name': 'bayes-sweep',
    'metric': {
        'name': 'val_accuracy',
        'goal': 'maximize'
    },
    'parameters': {
        'epochs': {
            'values': [5, 10] 
        },
        'embedding_size': {
            'values': [64, 128, 256] 
        },
        'num_hidden_layers': {
            'values': [1, 2, 3]
        },
        'hidden_layer_size': {
            'values': [64, 128, 256] 
        },
        'cell_type': {
            'values': ['rnn', 'gru'] 
        },
        'learning_rate': {
            'values': [1e-2, 5e-3, 1e-3, 5e-4] 
        },
        'optimizer': {
            'values': ['SGD', 'Adam'] 
        },
        'dropout': {
            'values': [0.0, 0.2, 0.3] 
        },
        'beam_width': {
            'values': [1] #[1, 3, 5]
        }
    }
}


In [19]:
def hyperparameter_search():
    run = wandb.init(project='hyperparam-search-enc-dec', reinit=True)
    config = wandb.config
    wandb.run.name = f'hl_{config.num_hidden_layers}_sz_{config.hidden_layer_size}_ct_{config.cell_type}_lr_{config.learning_rate}_opt_{config.optimizer}_do_{config.dropout}_bw_{config.beam_width}_em_{config.embedding_size}'
    
    encoder = EncoderRNN(input_lang.n_chars, config.embedding_size, config.hidden_layer_size, config.num_hidden_layers, dropout=config.dropout, cell_type=config.cell_type).to(device)
    decoder = DecoderRNN(config.hidden_layer_size, config.embedding_size, output_lang.n_chars, config.num_hidden_layers, dropout=config.dropout, cell_type=config.cell_type).to(device)

    if config.optimizer == 'SGD':
        optimizer = optim.SGD
    elif config.optimizer == 'Adam':
        optimizer = optim.Adam

    trainBatch(encoder, decoder, config.epochs, optimizer=optimizer, beam_width=config.beam_width, print_every=100, learning_rate=config.learning_rate, log=True)
    run.finish()

In [20]:
sweep_id = wandb.sweep(sweep_configuration, project='hyperparam-search-seq2seq')
wandb.agent(sweep_id, hyperparameter_search, count=15)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Create sweep with ID: 3fgeodv1
Sweep URL: https://wandb.ai/abdullah_010/hyperparam-search-seq2seq/sweeps/3fgeodv1


wandb: Agent Starting Run: p2hzajm1 with config:
wandb: 	beam_width: 1
wandb: 	cell_type: gru
wandb: 	dropout: 0.3
wandb: 	embedding_size: 128
wandb: 	epochs: 5
wandb: 	hidden_layer_size: 128
wandb: 	learning_rate: 0.01
wandb: 	num_hidden_layers: 3
wandb: 	optimizer: SGD
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: cs20b001 (abdullah_010). Use `wandb login --relogin` to force relogin
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


0m 1s (- 16m 19s) (100 0%) 2.2513
0m 3s (- 14m 21s) (200 0%) 2.5618
0m 5s (- 14m 11s) (300 0%) 2.9179
0m 6s (- 13m 50s) (400 0%) 2.9305
0m 8s (- 13m 44s) (500 0%) 2.9150
0m 9s (- 13m 35s) (600 1%) 2.8637
0m 11s (- 13m 27s) (700 1%) 2.8773
0m 12s (- 13m 22s) (800 1%) 2.7832
0m 14s (- 13m 23s) (900 1%) 2.7664
0m 16s (- 13m 22s) (1000 1%) 2.8304
0m 17s (- 13m 23s) (1100 2%) 2.7639
0m 19s (- 13m 24s) (1200 2%) 2.8745
0m 21s (- 13m 24s) (1300 2%) 2.7236
0m 23s (- 13m 35s) (1400 2%) 2.8114
0m 25s (- 13m 34s) (1500 2%) 2.7343
0m 26s (- 13m 28s) (1600 3%) 2.7377
0m 28s (- 13m 23s) (1700 3%) 2.7426
0m 29s (- 13m 17s) (1800 3%) 2.7399
0m 31s (- 13m 15s) (1900 3%) 2.7367
0m 32s (- 13m 15s) (2000 3%) 2.7490
0m 34s (- 13m 13s) (2100 4%) 2.7318
0m 36s (- 13m 19s) (2200 4%) 2.6546
0m 38s (- 13m 24s) (2300 4%) 2.6592
0m 40s (- 13m 31s) (2400 4%) 2.6476
0m 42s (- 13m 39s) (2500 4%) 2.6744
0m 45s (- 13m 45s) (2600 5%) 2.5436
0m 47s (- 13m 50s) (2700 5%) 2.5354
0m 49s (- 13m 52s) (2800 5%) 2.4817
0m 51s 

epoch,▁▃▅▆█
train_accuracy,▁▄▅▇█
train_loss,▅█▅▄▅▃▃▂▃▄▃▂▃▁▁▂▂▁▃▂▁▂▁▁▁▁▂▁▅▁▁▁▆▂▁▁▁▂▁▂
val_accuracy,▁▅▆▇█
epoch,5.0
train_accuracy,0.50089
train_loss,0.08639
val_accuracy,0.47546


...

wandb: Agent Starting Run: vscn718z with config:
wandb: 	beam_width: 1
wandb: 	cell_type: rnn
wandb: 	dropout: 0.3
wandb: 	embedding_size: 64
wandb: 	epochs: 10
wandb: 	hidden_layer_size: 256
wandb: 	learning_rate: 0.0005
wandb: 	num_hidden_layers: 2
wandb: 	optimizer: Adam
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


0m 1s (- 21m 53s) (100 0%) 2.6128
0m 2s (- 20m 14s) (200 0%) 2.9017
0m 3s (- 19m 59s) (300 0%) 2.8162
0m 4s (- 19m 33s) (400 0%) 2.8004
0m 5s (- 19m 10s) (500 0%) 2.7043
0m 6s (- 19m 26s) (600 0%) 2.8236
0m 8s (- 19m 48s) (700 0%) 2.8001
0m 9s (- 20m 3s) (800 0%) 2.6066
0m 11s (- 20m 22s) (900 0%) 2.6866
0m 12s (- 20m 33s) (1000 0%) 2.6157
0m 13s (- 20m 42s) (1100 1%) 2.6661
0m 15s (- 20m 52s) (1200 1%) 2.6333
0m 16s (- 20m 57s) (1300 1%) 2.5481
0m 17s (- 21m 4s) (1400 1%) 2.5724
0m 19s (- 21m 6s) (1500 1%) 2.5479
0m 20s (- 21m 8s) (1600 1%) 2.5351
0m 21s (- 21m 7s) (1700 1%) 2.5152
0m 23s (- 21m 9s) (1800 1%) 2.6793
0m 24s (- 21m 10s) (1900 1%) 2.5364
0m 25s (- 21m 9s) (2000 1%) 2.6774
0m 27s (- 21m 12s) (2100 2%) 2.5911
0m 28s (- 21m 12s) (2200 2%) 2.6243
0m 29s (- 21m 12s) (2300 2%) 2.6015
0m 31s (- 21m 11s) (2400 2%) 2.5104
0m 32s (- 21m 11s) (2500 2%) 2.6538
0m 33s (- 21m 8s) (2600 2%) 2.5931
0m 35s (- 21m 11s) (2700 2%) 2.6601
0m 36s (- 21m 9s) (2800 2%) 2.5525
0m 37s (- 21m 7s) 

wandb: Network error (ConnectionError), entering retry loop.


17m 41s (- 9m 23s) (65700 65%) 2.3104
17m 42s (- 9m 22s) (65800 65%) 2.4204
17m 44s (- 9m 20s) (65900 65%) 2.4038
17m 45s (- 9m 18s) (66000 65%) 2.5631
17m 46s (- 9m 16s) (66100 65%) 2.3481
17m 48s (- 9m 15s) (66200 65%) 2.3699
17m 49s (- 9m 13s) (66300 65%) 2.3768
17m 50s (- 9m 11s) (66400 66%) 2.4445
17m 52s (- 9m 9s) (66500 66%) 2.4566
17m 53s (- 9m 7s) (66600 66%) 2.4345
17m 54s (- 9m 6s) (66700 66%) 2.3066
17m 55s (- 9m 4s) (66800 66%) 2.4389
17m 57s (- 9m 2s) (66900 66%) 2.4120
17m 58s (- 9m 0s) (67000 66%) 2.4238
17m 59s (- 8m 59s) (67100 66%) 2.3820
18m 1s (- 8m 57s) (67200 66%) 2.4823
18m 2s (- 8m 55s) (67300 66%) 2.4569
18m 3s (- 8m 53s) (67400 66%) 2.3332
18m 4s (- 8m 51s) (67500 67%) 2.3885
18m 6s (- 8m 50s) (67600 67%) 2.3344
18m 7s (- 8m 48s) (67700 67%) 2.4807
18m 8s (- 8m 46s) (67800 67%) 2.5029
18m 10s (- 8m 44s) (67900 67%) 2.3865
18m 11s (- 8m 43s) (68000 67%) 2.4360
18m 12s (- 8m 41s) (68100 67%) 2.2997
18m 13s (- 8m 39s) (68200 67%) 2.4408
18m 15s (- 8m 37s) (68300

epoch,▁▂▃▃▄▅▆▆▇█
train_accuracy,▁▁▁▁▁▁▁▁▁▁
train_loss,▆▂▆▆▆▄▁▅▂█▃▅▄▃▄▄▁▅▄▃▄▆▄▄▅▃▂▅▅▁▄▇▅▃▄▃▂▃▄▇
val_accuracy,▁▁▁▁▁▁▁▁▁▁
epoch,10.0
train_accuracy,0.0
train_loss,2.03732
val_accuracy,0.0


In [27]:
best_config = {'cell_type': 'lstm', 'dropout': 0.0, 'embedding_size': 256, 'epochs': 10, 'hidden_layer_size': 256, 'learning_rate': 0.0005, 'num_hidden_layers': 3, 'optimizer': 'Adam', 'beam_width': 1}

encoder = EncoderRNN(input_lang.n_chars, best_config['embedding_size'], best_config['hidden_layer_size'], best_config['num_hidden_layers'], dropout=best_config['dropout'], cell_type=best_config['cell_type']).to(device)
decoder = DecoderRNN(best_config['hidden_layer_size'], best_config['embedding_size'], output_lang.n_chars, best_config['num_hidden_layers'], dropout=best_config['dropout'], cell_type=best_config['cell_type']).to(device)


In [28]:
optimizer = optim.SGD if best_config['optimizer'] == 'SGD' else optim.Adam

trainBatch(encoder, decoder, best_config['epochs'], optimizer=optimizer, beam_width=best_config['beam_width'], print_every=100, learning_rate=best_config['learning_rate'])

torch.save(encoder.state_dict(), 'vanilla_model/encoder.pt')
torch.save(decoder.state_dict(), 'vanilla_model/decoder.pt')

0m 5s (- 93m 17s) (100 0%) 2.7445
0m 11s (- 92m 52s) (200 0%) 2.8792
0m 16s (- 91m 49s) (300 0%) 2.9015
0m 22s (- 92m 30s) (400 0%) 2.7889
0m 28s (- 95m 29s) (500 0%) 2.8744
0m 34s (- 96m 15s) (600 0%) 2.8234
0m 39s (- 94m 11s) (700 0%) 2.8116
0m 44s (- 92m 57s) (800 0%) 2.8105
0m 49s (- 92m 3s) (900 0%) 2.7717
0m 55s (- 91m 27s) (1000 0%) 2.7221
1m 0s (- 91m 4s) (1100 1%) 2.7466
1m 5s (- 90m 45s) (1200 1%) 2.7345
1m 10s (- 90m 18s) (1300 1%) 2.7155
1m 16s (- 89m 56s) (1400 1%) 2.6209
1m 21s (- 89m 25s) (1500 1%) 2.5453
1m 26s (- 89m 3s) (1600 1%) 2.6531
1m 31s (- 88m 38s) (1700 1%) 2.5595
1m 36s (- 88m 18s) (1800 1%) 2.6022
1m 41s (- 87m 58s) (1900 1%) 2.5532
1m 46s (- 87m 37s) (2000 1%) 2.5591
1m 51s (- 87m 18s) (2100 2%) 2.4447
1m 56s (- 87m 2s) (2200 2%) 2.4290
2m 1s (- 86m 49s) (2300 2%) 2.3926
2m 6s (- 86m 33s) (2400 2%) 2.2921
2m 12s (- 86m 28s) (2500 2%) 2.4376
2m 17s (- 86m 13s) (2600 2%) 2.2690
2m 22s (- 86m 8s) (2700 2%) 2.3410
2m 27s (- 85m 53s) (2800 2%) 2.1541
2m 32s (- 8

In [31]:
print(evalAccuracy(encoder, decoder, valid_pairs))
print(evalAccuracy(encoder, decoder, test_pairs))

0.7067484662576687
0.4384765625


In [29]:
encoder.load_state_dict(torch.load('vanilla_model/encoder.pt'))
decoder.load_state_dict(torch.load('vanilla_model/decoder.pt'))

prediction_pairs = []

for pair in test_pairs:
    output_word = evaluate(encoder, decoder, pair[0])
    prediction_pairs.append((pair[0], output_word))

with open('predictions_vanilla/predictions.csv', 'w') as f:
    for pair in prediction_pairs:
        f.write(pair[0] + ',' + pair[1] + '\n')

